<a href="https://colab.research.google.com/github/neerajviswajith/CE888_Lab10/blob/main/MovieClassifier/MovieClf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Data retrieval and pre-processing

In [1]:
# Importing libraries and corpora

import nltk
import pickle
nltk.download('movie_reviews')
import numpy as np
import pandas as pd
from nltk.corpus import movie_reviews as mr

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


In [2]:
# Data pre-processing and creating pandas dataframe of reviews and lables

reviews = []
for fileid in mr.fileids():
    tag, filename = fileid.split('/')
    reviews.append((filename, tag, mr.raw(fileid)))

frdf = pd.DataFrame(reviews, columns=['filename', 'label', 'review'])
frdf.head()

,filename,label,review
0,cv000_29416.txt,neg,"plot : two teen couples go to a church party ,..."
1,cv001_19502.txt,neg,the happy bastard's quick movie review \ndamn ...
2,cv002_17424.txt,neg,it is movies like these that make a jaded movi...
3,cv003_12683.txt,neg,""" quest for camelot "" is warner bros . ' firs..."
4,cv004_12641.txt,neg,synopsis : a mentally unstable man undergoing ...


### 2. Splitting DataFrame into training and test set.

In [3]:
from sklearn.model_selection import train_test_split

X = frdf['review']
y = frdf['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=41)

### 3. Creating Pipeline for feature extraction and Classification models (Multinomial Naive Bayes, Linear SVC and Maximum Entropy)

In [4]:
# Importing the models
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

# Multinomial Naive Bayes Pipeline:
corpus_clf_nb = Pipeline([('tfidf', TfidfVectorizer(sublinear_tf=True, ngram_range=(2, 2))),('clf',MultinomialNB()),])

# LinearSVC Pipeline:
corpus_clf_lsvc = Pipeline([('tfidf', TfidfVectorizer(sublinear_tf=True)),('clf',SVC(kernel='linear',probability=True)),])

# Maximum Entropy Classifier Pipeline:
corpus_clf_me = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1, 2))),('clf', LogisticRegression()),])

# Ensamble Stacking Classifier Pipeline:
en_clf_stack = StackingClassifier(estimators=[('nb', corpus_clf_nb), ('lsvc', corpus_clf_lsvc), ('me', corpus_clf_me)],final_estimator=LogisticRegression(),cv=6)

In [5]:
# Saving Model
Pklname = "MovieClf.pkl"  

with open(Pklname, 'wb') as file:  
    pickle.dump(en_clf_stack, file)

#### 4. Fitting Ensamble Stacking Classifier Pipeline to training data and creating predict object.

In [6]:
en_clf_stack.fit(X_train, y_train)
STpred = en_clf_stack.predict(X_test)

#### 5. Checking performance of ESC

In [7]:
# Confusion Matrix of ESC

confusion_matrix(y_test,STpred)

array([[87,  7],
       [ 9, 97]])

In [8]:
# Classifcation Report of ESC

print(classification_report(y_test,STpred))

              precision    recall  f1-score   support

         neg       0.91      0.93      0.92        94
         pos       0.93      0.92      0.92       106

    accuracy                           0.92       200
   macro avg       0.92      0.92      0.92       200
weighted avg       0.92      0.92      0.92       200



In [9]:
# Accuracy Score of ESC

print(f'Accuracy of Ensamble Classifier Model: {metrics.accuracy_score(y_test,STpred)*100}%')

Accuracy of Ensamble Classifier Model: 92.0%
